In [1]:
import pandas as pd
# 과학적 표기 안하도록
pd.options.display.float_format = '{:.2f}'.format

import numpy as np
import datetime

---
### 증감 결과 확인을 위한 함수

In [2]:
def increasing(col):
    for i in range(len(col)):
        if(i == 0):
            temp = col[i]
        else:
            if(temp > col[i]):
                return 0
            temp = col[i]
    return 1 

def decreasing(col):
    for i in range(len(col)):
        if(i==0):
            temp = col[i]
        else:
            if(temp < col[i]):
                return 0
            temp = col[i]
    return 1

---
### selenium으로 크롤링


In [3]:
import requests
from bs4 import BeautifulSoup as bs

In [4]:
# code로 접근 가능한 url
code_url = "https://finance.naver.com/item/main.nhn?code="
# 종목 분석 url
coinfo_url = "https://finance.naver.com/item/coinfo.nhn?code="
# 종목 분석 url (!naver)
coinfo_url_2 = "https://navercomp.wisereport.co.kr/v2/company/c1050001.aspx?cmp_cd=%s&cn="
# 시가총액 목록 url
size_market_url_0 = "https://finance.naver.com/sise/sise_market_sum.nhn?sosok=0&page="

size_market_url_1 = "https://finance.naver.com/sise/sise_market_sum.nhn?sosok=1&page="

In [5]:
# 오늘의 시가총액 상위 400 code 목록 구하기

codeL = []
# 코스피 상위 300
for i in range(1,7):
    soup = bs(requests.get(size_market_url_0+str(i)).text, 'html.parser')
    tempL = [x.attrs["href"].split("code=")[1] for x in soup.select("table ")[1].select_one("tbody").select("a.tltle")]
    codeL += tempL
# 코스닥 상위 100
for i in range(1, 3):
    soup = bs(requests.get(size_market_url_1+str(i)).text, 'html.parser')
    tempL = [x.attrs["href"].split("code=")[1] for x in soup.select("table ")[1].select_one("tbody").select("a.tltle")]
    codeL += tempL
             

In [6]:
# 오늘의 시가총액 code 목록 저장 
todayDate = datetime.datetime.now().strftime("%Y_%b_%d")
f = open("./"+todayDate+"_시가총액_상위권.txt", 'w')
f.write("\n".join(codeL))
f.close()

# 오늘의 시가총액 code 목록 불러오기
useDate = "2020_Feb_10"
f = open("./"+useDate+"_시가총액_상위권.txt", "r")
codeL = [x.replace("\n", "") for x in f.readlines()]
f.close()

---

### 컨센서스 확인 함수

In [7]:
# 종목 이름 얻기
code = "005387"
soup = bs(requests.get(code_url + code).text, 'html.parser')
name = soup.select_one("#middle > div.h_company > div.wrap_company > h2 > a").text
name

'현대차2우B'

In [9]:
def Consensus(code):
    code = str(code)
        
    # 종목 이름 얻기
    soup = bs(requests.get(code_url + code).text, 'html.parser')
    name = soup.select_one("#middle > div.h_company > div.wrap_company > h2 > a").text
    
    # 컨센서스 테이블 구축
    consenUrl = "https://navercomp.wisereport.co.kr/company/ajax/c1050001_data.aspx?flag=2&cmp_cd="\
                + code + "&finGubun=MAIN&frq=0&chartType=svg"
    
    # 컨센서스 없는 경우 1
    if(requests.get(consenUrl).json()["JsonData"] == []):
        return 
    df_table = pd.DataFrame(requests.get(consenUrl).json()["JsonData"]).iloc[:, :-1]

    # 매출액 ~ EV/EBITDA 숫자형으로 변경
    df_table.iloc[:, 1:-1] = df_table.iloc[:,1:-1].applymap(lambda x: "0" if x == "" else x).applymap(lambda x: float(x.replace(",","")))
    
    # 컨센서스 없는 경우 2
    if(df_table.iloc[:, 1:-1].sum().sum() == 0):
        return
    
    # 증감 결과 df table로 확인
    result_row = [0]*12
    for inc in [1, 2, 3, 4, 5, 6, 9]:
        if(increasing(df_table.iloc[:, inc])):
            result_row[inc] = 1
    for dec in [7, 8, 10]:
        if(decreasing(df_table.iloc[:, dec])):
            result_row[dec] = 1

#     result_df = pd.DataFrame([result_row], columns = df_table.columns)
#     df_table.append(result_df, ignore_index = True)
    
     #증감 결과 매출액 / 영업이익 / per * 10 roe * 5
    formula = [0,10,1,10,1,1,1,10,1,5,1,0]
    score = np.dot(result_row,formula)
    
#     print(code, name, score)
    if(score > 20):
        print(code, name, score)
#         print(df_table.iloc[:, 1:-1])

---

## 시가총액 상위 400개의 컨센서스 확인

In [10]:
for i in range(400):
    Consensus(codeL[i])

051900 LG생활건강 23
018260 삼성에스디에스 23
316140 우리금융지주 24
271560 오리온 23
000120 CJ대한통운 21
282330 BGF리테일 21
012510 더존비즈온 23
030000 제일기획 22
001040 CJ 22
007310 오뚜기 21
181710 NHN 21
009420 한올바이오파마 21
284740 쿠쿠홈시스 28
280360 롯데제과 27
192080 더블유게임즈 21
336370 두산솔루스 28
298020 효성티앤씨 23
003230 삼양식품 23
286940 롯데정보통신 21
145720 덴티움 21
005180 빙그레 28
298050 효성첨단소재 21
336260 두산퓨얼셀 29
253450 스튜디오드래곤 21
036830 솔브레인 21
058470 리노공업 23
030190 NICE평가정보 23
035900 JYP Ent. 23
200130 콜마비앤에이치 23
064760 티씨케이 23
086450 동국제약 21
067160 아프리카TV 23
267980 매일유업 23
060250 NHN한국사이버결제 23
044340 위닉스 22
033290 코웰패션 33
